In [4]:
import geopandas as gpd
import pandas as pd

# 1. Shapefile 읽기
gdf = gpd.read_file('인도.shp')

# 2. 좌표계 설정 (없거나 잘못된 경우)
if gdf.crs is None:
    print("좌표계가 정의되지 않았습니다. WKID 5186으로 지정합니다.")
    gdf.set_crs(epsg=5186, inplace=True)
elif gdf.crs.to_epsg() != 5186:
    print(f"현재 좌표계: {gdf.crs}")
    print("좌표계를 WKID 5186으로 재지정합니다.")
    gdf.to_crs(epsg=5186, inplace=True)

# 3. 좌표계 변환 (TM -> WGS84)
gdf = gdf.to_crs(epsg=4326)

# 4. 중심점 계산
gdf['rep_point'] = gdf.geometry.representative_point()
gdf[['위도', '경도']] = gdf['rep_point'].apply(lambda p: pd.Series([p.y, p.x]))

# 결과 확인
print(gdf[['위도', '경도']].head())


좌표계가 정의되지 않았습니다. WKID 5186으로 지정합니다.
          위도          경도
0  37.475196  126.685533
1  37.474532  126.685289
2  37.471580  126.687159
3  37.472759  126.685074
4  37.473605  126.683281


In [31]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, GeometryCollection  # Point는 여기서 가져옴
import numpy as np

# 벡터화 (Shapely 2.x)
df['geometry'] = gpd.points_from_xy(df['경도'], df['위도'])
geo_df = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

print(geo_df.head())


              센서명                   시간         위도          경도      지오해시  \
0  M-IOT-00000007  2021-10-01 00:00:00  37.494364  126.726780  wydj79xs   
1  M-IOT-00000007  2021-10-01 00:00:01  37.494364  126.727123  wydj79xu   
2  M-IOT-00000007  2021-10-01 00:00:02  37.494364  126.727123  wydj79xu   
3  M-IOT-00000007  2021-10-01 00:00:03  37.494364  126.727123  wydj79xu   
4  M-IOT-00000007  2021-10-01 00:00:04  37.494192  126.727467  wydj7c85   

       시   군구    동  조도                    geometry  
0  인천광역시  부평구  부평동  13  POINT (126.72678 37.49436)  
1  인천광역시  부평구  부평동   3  POINT (126.72712 37.49436)  
2  인천광역시  부평구  부평동   3  POINT (126.72712 37.49436)  
3  인천광역시  부평구  부평동   3  POINT (126.72712 37.49436)  
4  인천광역시  부평구  부평동   3  POINT (126.72747 37.49419)  


In [11]:
import pandas as pd
import geopandas as gpd

df['geometry'] = gpd.points_from_xy(df['경도'], df['위도'])
geo_df = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

gdf['시간'] = None
gdf['조도'] = None

for emd_nm in gdf['sgg_nm'].unique():
    gdf_sub = gdf[gdf['sgg_nm'] == emd_nm]
    df_sub = geo_df[geo_df['군구'] == emd_nm]

    if df_sub.empty:
        print(f"{emd_nm}: df_sub에 데이터 없음, 건너뜀")
        continue

    if gdf_sub.crs != df_sub.crs:
        df_sub = df_sub.to_crs(gdf_sub.crs)

    nearest = gpd.sjoin_nearest(
        gdf_sub, 
        df_sub[['시간', '조도', 'geometry']], 
        how='left', 
        max_distance=0.01,
        lsuffix='_gdf', 
        rsuffix='_df'
    )

    gdf.loc[nearest.index, '시간'] = nearest['시간__df']
    gdf.loc[nearest.index, '조도'] = nearest['조도__df']

gdf


/home/basa/.conda/envs/py312_ljh/lib/python3.12/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/home/basa/.conda/envs/py312_ljh/lib/python3.12/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/home/basa/.conda/envs/py312_ljh/lib/python3.12/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/home/basa/.conda/envs/py312_ljh/lib/python3.12/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geogr

,objectid,indo_id,sido_nm,sgg_nm,emd_nm,edit_date,edit_code,st_area_sh,st_length_,geometry,rep_point,위도,경도,시간,조도
0,1,544.0,인천광역시,서구,가좌3동,2021-12-03,A,231.591857,123.258680,"POLYGON ((126.68575 37.47503, 126.68574 37.475...",POINT (126.68553 37.4752),37.475196,126.685533,2021-11-17 14:15:41,22786
1,2,542.0,인천광역시,서구,가좌3동,2021-12-03,A,300.798851,330.778248,"POLYGON ((126.68451 37.47387, 126.68451 37.473...",POINT (126.68529 37.47453),37.474532,126.685289,2021-11-17 14:15:41,22786
2,3,525.0,인천광역시,서구,가좌3동,2021-12-03,A,254.342128,153.750610,"POLYGON ((126.68738 37.47145, 126.68737 37.471...",POINT (126.68716 37.47158),37.471580,126.687159,2021-10-19 11:24:55,3903
3,4,526.0,인천광역시,서구,가좌3동,2021-12-03,A,1160.365578,666.283231,"POLYGON ((126.68654 37.47196, 126.68658 37.471...",POINT (126.68507 37.47276),37.472759,126.685074,2021-11-03 19:21:30,8
4,5,527.0,인천광역시,서구,가좌3동,2021-12-03,A,134.048600,83.401540,"POLYGON ((126.68347 37.47356, 126.68346 37.473...",POINT (126.68328 37.47361),37.473605,126.683281,2021-11-03 19:21:30,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14870,14870,14603.0,인천광역시,동구,화수2동,2021-12-03,A,94.798031,103.609890,"POLYGON ((126.63369 37.48253, 126.63368 37.482...",POINT (126.63354 37.48264),37.482639,126.633540,2021-10-11 19:36:13,4
14871,14872,14821.0,인천광역시,계양구,효성2동,2021-12-03,A,150.809219,93.936749,"POLYGON ((126.70536 37.52682, 126.70536 37.526...",POINT (126.70535 37.52693),37.526930,126.705348,2021-10-23 17:43:12,598
14872,14873,14867.0,인천광역시,계양구,효성2동,2021-12-03,A,663.671810,330.393697,"POLYGON ((126.70066 37.52803, 126.70002 37.527...",POINT (126.70065 37.52816),37.528156,126.700648,2021-10-29 07:48:18,13312
14873,14874,14868.0,인천광역시,계양구,효성2동,2021-12-03,A,641.428650,406.510166,"POLYGON ((126.71562 37.52799, 126.71616 37.527...",POINT (126.71548 37.5283),37.528302,126.715478,2021-10-11 21:23:32,42


In [26]:
# gdf 'emd_nm' 고유값
gdf_emd_nm = set(gdf['sgg_nm'].unique())
# geo_df '군구' 고유값
geo_df_gungu = set(geo_df['군구'].unique())

# gdf에는 있는데 geo_df에는 없는 값들
only_in_gdf = gdf_emd_nm - geo_df_gungu
print("gdf에는 있는데 geo_df에는 없는 읍면동명 (emd_nm):", only_in_gdf)

# geo_df에는 있는데 gdf에는 없는 값들
only_in_geo_df = geo_df_gungu - gdf_emd_nm
print("geo_df에는 있는데 gdf에는 없는 군구명:", only_in_geo_df)


gdf에는 있는데 geo_df에는 없는 읍면동명 (emd_nm): set()
geo_df에는 있는데 gdf에는 없는 군구명: {'강화군', '옹진군'}


In [32]:
import geopandas as gpd
import pandas as pd
import requests
import xmltodict
from datetime import datetime, timedelta
from shapely.geometry import Point
import math

class LCCParameter:
    def __init__(self):
        self.Re = 6371.00877  # 지도반경 (km)
        self.grid = 5.0        # 격자간격 (km)
        self.slat1 = 30.0      # 표준위도 1 (degree)
        self.slat2 = 60.0      # 표준위도 2 (degree)
        self.olon = 126.0      # 기준점 경도 (degree)
        self.olat = 38.0       # 기준점 위도 (degree)
        self.xo = 210 / self.grid  # 기준점 X 좌표
        self.yo = 675 / self.grid  # 기준점 Y 좌표
        self.first = False      # 초기화 여부

        # 초기화
        self.PI = math.asin(1.0) * 2.0
        self.DEGRAD = self.PI / 180.0
        self.RADDEG = 180.0 / self.PI
        self.re = self.Re / self.grid
        self.init_params()

    def init_params(self):
        slat1 = self.slat1 * self.DEGRAD
        slat2 = self.slat2 * self.DEGRAD
        olon = self.olon * self.DEGRAD
        olat = self.olat * self.DEGRAD

        sn = math.tan(self.PI * 0.25 + slat2 * 0.5) / math.tan(self.PI * 0.25 + slat1 * 0.5)
        sn = math.log(math.cos(slat1) / math.cos(slat2)) / math.log(sn)
        sf = math.tan(self.PI * 0.25 + slat1 * 0.5)
        sf = math.pow(sf, sn) * math.cos(slat1) / sn
        ro = math.tan(self.PI * 0.25 + olat * 0.5)
        ro = self.re * sf / math.pow(ro, sn)

        self.sn = sn
        self.sf = sf
        self.ro = ro
        self.olon_rad = olon
        self.olat_rad = olat
        self.first = True

def lamcproj(lon, lat, code, map):
    if code == 0:  # 위경도 -> 격자
        ra = math.tan(map.PI * 0.25 + lat * map.DEGRAD * 0.5)
        ra = map.re * map.sf / math.pow(ra, map.sn)
        theta = lon * map.DEGRAD - map.olon_rad

        if theta > map.PI:
            theta -= 2.0 * map.PI
        if theta < -map.PI:
            theta += 2.0 * map.PI

        theta *= map.sn
        x = int(ra * math.sin(theta) + map.xo + 1.5)
        y = int(map.ro - ra * math.cos(theta) + map.yo + 1.5)

        return x, y

    elif code == 1:  # 격자 -> 위경도
        xn = lon - map.xo
        yn = map.ro - lat + map.yo
        ra = math.sqrt(xn * xn + yn * yn)

        if map.sn < 0.0:
            ra = -ra

        alat = math.pow((map.re * map.sf / ra), (1.0 / map.sn))
        alat = 2.0 * math.atan(alat) - map.PI * 0.5

        if abs(xn) <= 0.0:
            theta = 0.0
        else:
            if abs(yn) <= 0.0:
                theta = map.PI * 0.5
                if xn < 0.0:
                    theta = -theta
            else:
                theta = math.atan2(xn, yn)

        alon = theta / map.sn + map.olon_rad
        return alon * map.RADDEG, alat * map.RADDEG

# SHP 파일 불러오기
gdf = gpd.read_file('C:/Users/user/Desktop/인도_조도_경사도.shp')
gdf=gdf.head()

# 2. 좌표계 확인 및 지정
if gdf.crs is None:
    print("좌표계가 정의되지 않았습니다. WKID 5186으로 지정합니다.")
    gdf.set_crs(epsg=5186, inplace=True)
else:
    print(f"현재 좌표계: {gdf.crs}")
    if gdf.crs != 'EPSG:5186':
        print("좌표계를 WKID 5186으로 재지정합니다.")
        gdf.set_crs(epsg=5186, inplace=True, allow_override=True)

# 좌표계 변환 (TM -> WGS84)
gdf = gdf.to_crs(epsg=4326)



# 격자 좌표 변환 (WGS84 -> 기상청 격자)
map = LCCParameter()
gdf[['nx', 'ny']] = gdf.apply(lambda row: pd.Series(lamcproj(row['경도'], row['위도'], 0, map)), axis=1)

def get_base_time():
    now = datetime.now()
    # 기준 발표 시각 리스트 (시간, 분)
    base_times = [
        (2, 0), (5, 0), (8, 0), (11, 0),
        (14, 0), (17, 0), (20, 0), (23, 0)
    ]
    
    # API 제공 시각 = Base_time + 10분
    api_times = [(h, 10) for (h, m) in base_times]

    # 현재 시각이 API 제공 시간 이후인 가장 최신 Base_time을 찾기
    latest_base = None
    for i in range(len(api_times)):
        api_hour, api_min = api_times[i]
        api_datetime = now.replace(hour=api_hour, minute=api_min, second=0, microsecond=0)
        if now >= api_datetime:
            latest_base = base_times[i]
        else:
            break

    # 만약 최신 base_time이 없으면 (예: 00:00 ~ 02:09 사이)
    if latest_base is None:
        base_date = (now - timedelta(days=1)).strftime('%Y%m%d')
        base_time = '2300'
    else:
        base_date = now.strftime('%Y%m%d')
        base_time = f'{latest_base[0]:02}00'

    return base_date, base_time
    
# 날씨 데이터 가져오기 함수
def get_weather(nx, ny):
    base_date, base_time = get_base_time()
    API_BASE_URL = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"
    params = {
        'pageNo': 1,
        'numOfRows': 1000,
        'dataType': 'XML',
        'base_date': base_date,
        'base_time': base_time,
        'nx': nx,
        'ny': ny,
        'authKey': "rP3MkYGETA69zJGBhPwOnA"  # 실제 인증키로 변경 필요
    }

    try:
        response = requests.get(API_BASE_URL, params=params)
        response.raise_for_status()
        data = xmltodict.parse(response.text)
        items = data['response']['body']['items']['item']
        weather = {}
        for item in items:
            category = item['category']
            value = item['fcstValue']
            weather[category] = value
        return weather
        
    except Exception as e:
        print(f"API 요청 오류 (nx: {nx}, ny: {ny}): {e}")
        return {}

# 날씨 데이터 추가
gdf['weather'] = gdf.apply(lambda row: get_weather(row['nx'], row['ny']), axis=1)

# 결과 출력 (확인용)
print(gdf[['위도', '경도', 'nx', 'ny', 'weather']].head())


현재 좌표계: EPSG:5186
          위도          경도  nx   ny  \
0  37.475196  126.685533  55  125   
1  37.474532  126.685289  55  125   
2  37.471580  126.687159  55  125   
3  37.472759  126.685074  55  125   
4  37.473605  126.683281  55  125   

                                             weather  
0  {'TMP': '17', 'UUU': '0.7', 'VVV': '0.9', 'VEC...  
1  {'TMP': '17', 'UUU': '0.7', 'VVV': '0.9', 'VEC...  
2  {'TMP': '17', 'UUU': '0.7', 'VVV': '0.9', 'VEC...  
3  {'TMP': '17', 'UUU': '0.7', 'VVV': '0.9', 'VEC...  
4  {'TMP': '17', 'UUU': '0.7', 'VVV': '0.9', 'VEC...  
